In [1]:
import json
import random
import numpy as np
import tensorflow as tf
import pandas as pd
from dask import dataframe as dd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

## Preparing the data

In [ ]:
transactions = dd.read_csv('data/transactions_train.csv')[["customer_id", "article_id"]]
customer_purchase_number = transactions.groupby("customer_id").size()
customer_purchase_number = customer_purchase_number[customer_purchase_number > 3].compute()
transactions = transactions[transactions["customer_id"].isin(customer_purchase_number.keys())].reset_index(drop=True)
c_ids = transactions.customer_id.unique()
number_of_customer = len(c_ids)
customer_encoding = {c_id: i for i, c_id in enumerate(c_ids)}
p_ids = transactions.article_id.unique()
number_of_products = len(p_ids)
product_encoding = {p_id: i for i, p_id in enumerate(p_ids)}
with open('model_data/customer_id_encoding.json', 'w') as fp:
    json.dump(customer_encoding, fp)

with open('model_data/product_id_encoding.json', 'w') as fp:
    json.dump(product_encoding, fp)

transactions.customer_id = transactions.customer_id.map(customer_encoding)
transactions.article_id = transactions.article_id.map(product_encoding)
p_ids = list(product_encoding.values())
del c_ids, product_encoding, customer_encoding

In [ ]:
train, test = transactions.random_split([0.9, 0.1], random_state=43)
df = test.merge(train[["customer_id"]], on=["customer_id"], how="outer", indicator=True)
train = dd.concat([train, df[df['_merge'] == 'left_only'][["customer_id", "article_id"]]], axis=0, ignore_index=True, interleave_partitions=True, ignore_order=True)
df = df[df['_merge'] == 'both'][["customer_id"]].drop_duplicates()
test = test.merge(df, how="inner", on="customer_id")
steps_per_epoch = len(train)

In [ ]:
def prepare_data(transactions, p_ids):
    transactions = transactions.groupby(["customer_id"])['article_id']\
                            .apply(lambda x: list(x), meta=("article_ids",object))\
                            .reset_index().compute().drop_duplicates(subset=["customer_id"])

    transactions["hist_len"] = transactions.article_ids.apply(lambda x: 128 if len(x)>128 else len(x))
    transactions["prod_ids"] = transactions.apply(lambda x: random.sample(x.article_ids, x.hist_len), axis=1)
    transactions["not_prods"] = transactions.prod_ids.apply(lambda x: [p_id for p_id in random.sample(p_ids, 256) if p_id not in x][:128])
    transactions = transactions[["customer_id", "prod_ids", "not_prods"]]
    return transactions

train = prepare_data(train, p_ids)
test = prepare_data(test, p_ids)
train.to_pickle('data/train.csv', index=False)
test.to_pickle('data/test.csv', index=False)
print(train.shape, test.shape)
del p_ids, transactions

## Data Loader

In [2]:
class Generator():
    def __init__(self, data, positive_sample_length, negative_sample_length, batch_size):
        self.data = data
        self.positive_sample_length = positive_sample_length
        self.negative_sample_length = negative_sample_length
        self.sample_length = positive_sample_length + negative_sample_length
        if batch_size % self.sample_length != 0:
            raise ValueError("batch_size must be divisible by sum of positive_sample_length and negative_sample_length")


    def user_info_generator(self):
        for i, row in self.data[["customer_id","prod_ids","not_prods"]].iterrows():
            pids = np.asarray(random.sample(row["prod_ids"], self.positive_sample_length) + random.sample(row["not_prods"], self.negative_sample_length))
            labels = np.asarray([1]*self.positive_sample_length + [0]*self.negative_sample_length)
            indices = np.arange(self.sample_length)
            np.random.shuffle(indices)
            yield tf.convert_to_tensor(np.asarray([pids[indices],labels[indices]], dtype=np.int32), dtype=tf.int32)

    def user_id_generator(self):
        for i, row in self.data[["customer_id"]].iterrows():
            customer_id = [row["customer_id"]] * self.sample_length
            yield tf.convert_to_tensor(customer_id, dtype=tf.int32)

    def shuffle(self):
        self.data = shuffle(self.data)

class TGenerator (Generator):
    def __init__(self, data, positive_sample_length, negative_sample_length, batch_size):
        super().__init__(data, positive_sample_length, negative_sample_length, batch_size)
        self.mini_batch = int(batch_size / self.sample_length)
        self.batch = batch_size
        self.user_info_loader = tf.data.Dataset.from_generator(
            self.user_info_generator, output_types=tf.int32).batch(self.mini_batch, drop_remainder=True).repeat()
        self.user_id_loader = tf.data.Dataset.from_generator(
            self.user_id_generator, output_types=tf.int32).batch(self.mini_batch, drop_remainder=True).repeat()

    def get(self):
        for c_ids, info in zip(self.user_id_loader, self.user_info_loader):
            input = tf.stack([tf.reshape(c_ids, self.batch), tf.reshape(info[:,0,:], self.batch)], axis=1)
            label = tf.reshape(info[:,1,:], self.batch)
            yield input, label

class VGenerator (Generator):
    def __init__(self, data, positive_sample_length, negative_sample_length, batch_size):
        super().__init__(data, positive_sample_length, negative_sample_length, batch_size)
        self.mini_batch = int(batch_size / self.sample_length)
        self.batch = batch_size
        self.user_info_loader = tf.data.Dataset.from_generator(
            self.user_info_generator, output_types=tf.int32).batch(self.mini_batch, drop_remainder=True).repeat()
        self.user_id_loader = tf.data.Dataset.from_generator(
            self.user_id_generator, output_types=tf.int32).batch(self.mini_batch, drop_remainder=True).repeat()

    def get(self):
        for c_ids, info in zip(self.user_id_loader, self.user_info_loader):
            try:
                input = tf.stack([tf.reshape(c_ids, self.batch), tf.reshape(info[:,0,:], self.batch)], axis=1)
                label = tf.reshape(info[:,1,:], self.batch)
                yield input, label
            except:
                print("error")
                yield None, None

In [3]:
batch_size = 256
train_data = pd.read_pickle("data/train.csv")
test_data = pd.read_pickle("data/test.csv")
train_loader = TGenerator(train_data, 1, 1, batch_size)

validation_loader = VGenerator(test_data, 1, 1, batch_size)
train = train_loader.get()
val = validation_loader.get()


batch_size = 512
steps_per_epoch = len(train)
 = pd.read_pickle("data/train.csv")
test_data = pd.read_pickle("data/test.csv")
train_loader = TGenerator(train, 1, 1, batch_size)
validation_loader = VGenerator(test, 1, 1, batch_size)
train_generator = train_loader.get()
val_generator = validation_loader.get

2022-04-08 21:45:56.615878: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Matrix Factorization Model (GMF)

In [ ]:
class RecommenderNet(tf.keras.Model):
    def __init__(self, num_users, num_prods, embedding_size, **kwargs):
        super(RecommenderNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_prods = num_prods
        self.embedding_size = embedding_size
        self.user_embedding = tf.keras.layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer= tf.keras.regularizers.l2(1e-6),
        )
        self.user_bias = tf.keras.layers.Embedding(num_users, 1)
        self.prod_embedding = tf.keras.layers.Embedding(
            num_prods,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer= tf.keras.regularizers.l2(1e-6),
        )
        self.prod_bias = tf.keras.layers.Embedding(num_prods, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        prod_vector = self.prod_embedding(inputs[:, 1])
        prod_bias = self.prod_bias(inputs[:, 1])
        dot_user_prod = tf.tensordot(user_vector, prod_vector, 2)
        x = dot_user_prod + user_bias + prod_bias

        return tf.nn.sigmoid(x)


In [ ]:
model = RecommenderNet(number_of_customer, number_of_products, 128)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [ ]:
model.fit(train, epochs=1, steps_per_epoch= steps_per_epoch // batch_size, validation_data = val)

In [ ]:
next(iter(train))

StopIteration: 